In [1]:
import sys

import matplotlib.pyplot as plt

try:
    __file__
except:
    sys.argv = [sys.argv[0],'--dynet-gpu', '--dynet-mem', '4000']
import matplotlib.pyplot as plt
import dynet as dy


In [2]:
def parse_level(file_name):
    with open(file_name) as input_file:
        rows = [list(line.rstrip()) for line in input_file]
        column_len = len(rows[0])
        print len(rows)
        data = ['*start*','(']
        flip = True
        for c in range(column_len):
            if flip:
                for r in range(len(rows)):
                    data.append(rows[r][c])
                data.append(')')
            else:
                for r in range(len(rows)-1,-1,-1):
                    data.append(rows[r][c])
                data.append('(')
            flip = not flip
            if c % 10 == 0:
                data += ['^']*(int(c/10))
        data.append('*end*')
    return data


levels = ['TheVGLC-master/Super Mario Bros/Processed/mario-1-1.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-1-2.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-1-3.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-2-1.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-3-1.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-3-3.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-4-1.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-4-2.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-5-1.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-5-3.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-6-1.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-6-2.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-6-3.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-7-1.txt',
    'TheVGLC-master/Super Mario Bros/Processed/mario-8-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World1-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World1-2.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World1-3.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World2-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World2-2.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World2-3.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World3-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World3-3.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World4-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World4-2.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World4-3.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World5-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World5-2.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World6-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World6-3.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-World8-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-WorldA-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-WorldA-3.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-WorldB-1.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-WorldB-3.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-WorldC-2.txt',
    'TheVGLC-master/Super Mario Bros 2 (Japan)/Processed/SuperMarioBros2(J)-WorldD-1.txt',    
]


levels = [parse_level(l) for l in levels]

14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
12
14
15
15
14
14
16
15
15
14
14
14
15
15
13
15
14
15
14
15
15
15


In [4]:
v = [set(level) for level in levels]
v = reduce(lambda x,y: x | y,v)

v2i = {v:i for i,v in enumerate(sorted(v))}
i2v = {i:v for v,i in v2i.items()}

embeddings_size = len(v2i)
encoder_state_size = 512

encoding_size = 64

dropout_max = 0.8
dropout_min = 0.0
dropout_midpoint = -100
dropout_steepness = 0.125

decoder_state_size = 512
layers = 2
total = 0

model = dy.Model()

encoder_embedding = model.add_lookup_parameters((len(v2i), embeddings_size))
encoder_forward_rnn = dy.LSTMBuilder(layers, embeddings_size, encoder_state_size, model)
encoder_backward_rnn = dy.LSTMBuilder(layers, embeddings_size, encoder_state_size, model)
#encoder_backward_rnn.set_dropout(dropout_min)

encoder_w_mean = model.add_parameters((encoding_size, encoder_state_size*2)) 
encoder_b_mean = model.add_parameters((encoding_size))

encoder_w_logvar = model.add_parameters((encoding_size, encoder_state_size*2)) 
encoder_b_logvar = model.add_parameters((encoding_size))

decoder_rnn = dy.LSTMBuilder(layers, encoding_size+embeddings_size, decoder_state_size, model)
decoder_rnn.set_dropout(dropout_min)


decoder_w = model.add_parameters((len(v2i), decoder_state_size)) 
decoder_b = model.add_parameters((len(v2i)))



In [19]:
import numpy as np
    
trainer = dy.RMSPropTrainer(model)

def run_rnn(init_state, input_vecs):
    s = init_state

    states = s.add_inputs(input_vecs)
    rnn_outputs = [s.output() for s in states]
    return rnn_outputs
    
def encode_string(embedded_string,rnn):
    initial_state = rnn.initial_state()

    # run_rnn returns all the hidden state of all the slices of the RNN
    hidden_states = run_rnn(initial_state, embedded_string)

    return hidden_states

def embed_string(embeddings, string):
    return [embeddings[tok] for tok in string]

def get_loss(level,loss_annealing=1.0):  
    dy.renew_cg()      
    states = []
    
    embedded_level = embed_string(encoder_embedding,level)
    
    forward_encoding = encode_string(embedded_level,encoder_forward_rnn)
    backward_encoding = encode_string(embedded_level[::-1],encoder_backward_rnn)
    
    encoder_w_mean_ = dy.parameter(encoder_w_mean)
    encoder_b_mean_ = dy.parameter(encoder_b_mean)
    
    encoder_w_logvar_ = dy.parameter(encoder_w_logvar)
    encoder_b_logvar_ = dy.parameter(encoder_b_logvar)
    
    encoder_mean = encoder_w_mean_*dy.concatenate([forward_encoding[-1],backward_encoding[-1]])+encoder_b_mean_
    encoder_logvar = encoder_w_logvar_*dy.concatenate([forward_encoding[-1],backward_encoding[-1]])+encoder_b_logvar_
    
    epsilon = dy.noise(dy.vecInput(encoding_size),1.0)
    encoded_state = dy.tanh(encoder_mean + dy.cmult(dy.exp(encoder_logvar / 2.0), epsilon))
     
    decoder_state = decoder_rnn.initial_state()
    decoder_state = decoder_state.add_input(dy.concatenate([encoded_state,encoder_embedding[level[0]]]))
    
    output_w = dy.parameter(decoder_w)
    output_b = dy.parameter(decoder_b)
    loss = []
    for tile in level[1:]:
        rnn_output = decoder_state.output()
        probs = dy.softmax(output_w * rnn_output + output_b)
        loss.append(-dy.log(dy.pick(probs, tile)))
        decoder_state = decoder_state.add_input(dy.concatenate([encoded_state,encoder_embedding[tile]]))
    decoder_state = decoder_state.add_input(dy.concatenate([encoded_state,encoder_embedding[tile]]))
    
    kl_loss = - 0.5 *dy.sum_elems(1 + encoder_logvar - dy.square(encoder_mean) - dy.exp(encoder_logvar))
    
    print kl_loss.value()
    
    loss = dy.esum(loss)+kl_loss*loss_annealing
    return loss
    
def train(level,loss_annealing=1.0):  
    loss = get_loss(level,loss_annealing)
    loss.backward()
    trainer.update()
    return loss
    

In [ ]:
import random


random.shuffle(levels)
training_set = levels[:int(len(levels)*0.8)]
validation_set = levels[int(len(levels)*0.8):]
best = float('inf')



kl_max = 2.0
kl_min = 0.0
kl_midpoint = 50
kl_steepness = 0.125


for ii in range(2000):
    total += 1
    dropout = dropout_min + dropout_max/(1.0+np.exp(-dropout_steepness*(ii-dropout_midpoint)))
    kl = kl_min + kl_max/(1.0+np.exp(-kl_steepness*(ii-kl_midpoint)))
    
    print 'epoch', total,kl
    #encoder_forward_rnn.set_dropout(dropout)
    decoder_rnn.set_dropout(dropout)

    random.shuffle(training_set)
    print 'TRAINING'
    total_train = 0
    total_size = 0
    for level in training_set:
        total_size += len(level)
        total_train += train([v2i[tile] for tile in level],kl).value()
    print total_train/float(total_size)
    print 'VALIDATION'
    total_validation = 0
    total_size = 0
    
    for level in validation_set:
        val_loss =  get_loss([v2i[tile] for tile in level],kl).value()
        total_size += len(level)
        total_validation += val_loss
    total_validation = total_validation/float(total_size)
    print total_validation
    if total_validation < best:
        best = total_validation
        model.save('{}_{}_{}_{}_{}_{}.model'.format(encoder_state_size,decoder_state_size,layers,dropout_max,encoding_size,total_validation))

epoch 15 0.00385346932665
TRAINING
10.9191036224
10.5480699539
10.3432416916
10.1097297668
10.3153553009
10.7473583221
10.7198705673
11.5105371475
12.9115877151
13.5326557159
15.022860527
15.785235405
16.5165481567
18.1645641327
17.6555023193
18.1155662537
18.4831943512
20.258441925
22.4704647064
23.7031421661
23.1587638855
23.4784221649
22.965511322
21.0526981354
23.7891082764
25.1963748932
25.643163681
24.2089614868
24.455165863
1.03072027619
VALIDATION
25.0137233734
25.0126781464
25.0129184723
25.0123729706
25.0137195587
25.0134048462
25.0127658844
25.013710022
1.06323936805
epoch 16 0.00436543289069
TRAINING
25.0126781464
25.090883255
24.3426437378
24.4013500214
23.1211242676
24.0420055389
25.4293727875
26.9511451721
26.9395923615
27.1704483032
27.7935218811
27.0923595428
27.145029068
25.9347648621
26.3763885498
26.2804679871
27.8457298279
28.5434494019
29.2114448547
29.9010334015
29.7951412201
28.9142608643
28.8429832458
29.840139389
29.7727375031
29.7037143707
31.1793327332
32.36

In [ ]:
import numpy.random

def encode(level):
    dy.renew_cg() 
    embedded_level = embed_string(encoder_embedding,level)
    forward_encoding = encode_string(embedded_level,encoder_forward_rnn)
    backward_encoding = encode_string(embedded_level[::-1],encoder_backward_rnn)
    encoder_w_ = dy.parameter(encoder_w)
    encoder_b_ = dy.parameter(encoder_b)
    encoded_state = dy.tanh(encoder_w_*dy.concatenate([forward_encoding[-1],backward_encoding[-1]])+encoder_b_) 
   
    return encoded_state.value()


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def generate(encoding,max_length=7000,temperature=1.0):
    dy.renew_cg() 
    encoding = dy.inputTensor(encoding)
    decoder_state = decoder_rnn.initial_state()
    decoder_state = decoder_state.add_input(dy.concatenate([encoding,
                                               encoder_embedding[v2i['*start*']]]))
    
    decoded = []
    
    output_w = dy.parameter(decoder_w)
    output_b = dy.parameter(decoder_b)
    for ii in xrange(max_length):
        rnn_output = decoder_state.output()
        probs = dy.softmax(output_w * rnn_output + output_b)
        sampled = sample(probs.value(),temperature)
        decoded.append(sampled)
        decoder_state = decoder_state.add_input(dy.concatenate([encoding,
                                                                encoder_embedding[sampled]]))
        if i2v[sampled] == '*end*':
            break
            
    return decoded

level = levels[1]



encoded = encode([v2i[tile] for tile in level])    
temp = 1.0

for ii in range(50):
    generated = generate(encoded,7000,temp)

    generated_str = [i2v[tok] for tok in generated]
    direction = -1
    current = 0
    level = []
    col = None
    for tok in generated_str:
        if tok == '(':
            if col:
                level.append(col)
            col = ['-']*14
            direction = 1
            current = 0
        elif tok == ')':
            if col:
                level.append(col)
            col = ['-']*14
            direction = -1
            current = 13
        elif tok == '^':
            pass
        elif tok == '*end*':
            pass
        else:
            if current < 0:
                current = 0
            if current > 13:
                current = 13
            col[current] = tok

            current += direction

    level_oriented = [list('-'*len(level)) for ii in range(len(col))]
    for ii in range(len(level)):
        for jj in range(len(level[ii])):

            level_oriented[jj][ii] = level[ii][jj]

    for row in level_oriented:
        print ''.join(row)
    

In [ ]:



import matplotlib.pyplot as plt

plt.hist(np.array(encoded),250)
plt.show()

for level in levels:
    print len(level)